In [39]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import AzureChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from openai import AzureOpenAI
import os

In [35]:
load_dotenv()

True

In [36]:
client = AzureOpenAI(azure_endpoint=os.getenv("azure_endpoint"), api_key=os.getenv("AZURE_OPENAI_API_KEY"),api_version="2024-12-01-preview" ) # Replace with the appropriate version
result = client.chat.completions.create(model="gpt-35-turbo", messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "what is the capital of India?",
        }])

In [45]:
model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("azure_endpoint"),
    api_version="2024-12-01-preview",  # Replace with the appropriate version
)

In [41]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [42]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [43]:
# Create graph

graph = StateGraph(LLMState)

# add nodes

graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [50]:
# execute

initial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'How far is moon from the earth?', 'answer': 'The average distance between the Earth and the Moon is about 384,400 kilometers (238,855 miles).'}
